In [8]:
import requests
import json
import pandas as pd
import time
import pickle

pd.options.display.max_rows = 9999

In [3]:
class Stock():
    def __init__(self, symbol,api_key):
        self.symbol=symbol
        self.api_key=api_key
        self.api_output=self.get_api_data()
        self.weight=1
        
    def get_api_data(self):
        """make api call"""
        #Would want to handle non-200 responses 
        r=requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+self.symbol+'&outputsize=full&apikey='+self.api_key)
        return r
    
    def get_stock_df(self):
        """Create a dataframe indexed by day wwhich has share price info"""
        stock_hist=json.loads(self.api_output.content)
        stock_df=pd.DataFrame(stock_hist['Time Series (Daily)']).transpose()
        stock_df=stock_df.astype(float)
        stock_df['9. symbol']=stock_hist['Meta Data']['2. Symbol']+' '
        return stock_df
    

class Index():
    #Need to do the API calls in the index because it's limited to 5/minute and 500/day
    def __init__(self):
        self.sp500_df=self.get_sp_df()
        self.sp500_prices={}
        self.api_all_stocks()
    def get_sp_df(self):
        #Need a better source for SP500 financials like mkcp
        sp500=requests.get('https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.json')
        sp500_df=pd.DataFrame(json.loads(sp500.content))
        sp500_df['Shares']=sp500_df['Market Cap']/sp500_df['Price']
        return sp500_df
    def sample_sp(self,num_sample):
        return self.sp500_df.sample(num_sample)
    
    def api_all_stocks(self):
        symb_list=self.sp500_df['Symbol'].tolist()
        for i in symb_list:
            self.sp500_prices[i]=Stock(i, api_key)
            time.sleep(15)

    
    
class Portfolio(Index):
    def __init__(self, symbol_list):
        self.symbol_list=symbol_list
        self.Stocks=self.get_all_stocks()
    def get_all_stocks(self):
        Stocks=[]
        for symbol in self.symbol_list:
            Stocks.append(Stock(symbol,api_key))
        return Stocks
    def total_portfolio_df(self):
        loop_check=0
        for stock in self.Stocks:
            if loop_check==0:
                agg_df=stock.weight*stock.get_stock_df()
            else:
                agg_df+=stock.get_stock_df()
        

In [ ]:
sp500.sp500_prices['XYL']=Stock('XYL',api_key)
sp500.sp500_prices['YUM']=Stock('YUM',api_key)
sp500.sp500_prices['ZBH']=Stock('ZBH',api_key)
sp500.sp500_prices['ZION']=Stock('ZION',api_key)
sp500.sp500_prices['ZTS']=Stock('ZTS',api_key)

In [ ]:
pickle.dump( sp500, open( "2019-06-01_save.p", "wb" ))

In [9]:
sp500 = pickle.load( open( "2019-06-01_save.p", "rb" ) )

In [ ]:
sp500.sp500_df

In [ ]:
def return_2018(symbol):
    shares=1000/float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)']['2018-01-02']['5. adjusted close'])
    return shares*float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)']['2018-12-31']['5. adjusted close'])

In [ ]:
pd.DataFrame(json.loads(sp500.sp500_prices['WYN'].api_output.content)['Time Series (Daily)']).transpose()

In [25]:
class Performance():
    def __init__(self,num_stocks=0, stock_list=None,start_date='2019-01-02', end_date='2019-05-28'):
        self.num_stock=num_stocks
        self.start_date=start_date
        self.end_date=end_date
        
        if stock_list is None:
            self.stock_list=self.get_stock_list(num_stocks)
        else:
            self.stock_list=stock_list
        
        self.stock_weights=self.get_stock_weights(self.stock_list)
        
        
    def get_adj_price(self, symbol, date=None):
        if date is None:
            date=self.start_date
        try:
            return float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][date]['5. adjusted close'])
        except:
            return 0
    def get_nom_price(self,symbol,date=None):
        if date is None:
            date=self.start_date
        try: 
            return float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][date]['4. close'])
        except:
            return 0
        
    def get_mkt_cap(self,symbol,date=None):
        if date is None:
            date=self.start_date
        try:
            return sp500.sp500_df[sp500.sp500_df['Symbol']==symbol]['Shares'].tolist()[0]*z.get_nom_price(symbol,date)
        except:
            return 0
    
    def get_total_return(self,symbol,start_date, end_date,weight=1000.00):
        try:
            shares=weight/float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][start_date]['5. adjusted close'])
            return shares*float(json.loads(sp500.sp500_prices[symbol].api_output.content)['Time Series (Daily)'][end_date]['5. adjusted close'])
        except:
            return 0
    def get_stock_weights(self,stock_list):
        weights={}
        tot_cap=1
        for i in stock_list:
            tot_cap+=self.get_mkt_cap(i)
        for i in stock_list:
            weights[i]=self.get_mkt_cap(i)/tot_cap
        return weights
        
    def portfolio_return(self):
        port_return=0
        for i in self.stock_list:
            port_return+=self.get_total_return(i,self.start_date,self.end_date,self.stock_weights[i])
        return port_return
        
    def get_stock_list(self,num_stocks):
        stock_list=[]
        for i in range(num_stocks):
            symbol=sample_stock(stock_list)
            stock_list.append(symbol)
        return stock_list
    
    def sample_stock(self, stock_list):
        stock_sample=sp500.sp500_df.sample(1)['Symbol'].tolist()[0]
        if stock_sample not in stock_list:
            try: 
                test=json.loads(sp500.sp500_prices[stock_sample].api_output.content)['Time Series (Daily)'][self.start_date]
                test2=json.loads(sp500.sp500_prices[stock_sample].api_output.content)['Time Series (Daily)'][self.end_date]
                return stock_sample
            except:
                return sample_stock(stock_list)
        else:
            return sample_stock(stock_list)

In [38]:
z=Performance(stock_list=sp500.sp500_df['Symbol'].tolist())
reduced_index=sp500.sp500_df['Symbol'].tolist()
for i in z.stock_weights:
    if z.stock_weights[i]==0:
        reduced_index.remove(i)

In [53]:
output={}
for i in reduced_index:
    index_num=reduced_index.index(i)
    stock_list=reduced_index[:index_num]+reduced_index[index_num+1:]
    z=Performance(stock_list=stock_list)
    print(i, z.portfolio_return())
    output[i]=z.portfolio_return()
#print(sp500.sp500_df[sp500.sp500_df['Symbol']=='AMZN']['Shares'].tolist()[0]*z.get_nom_price('AMZN'))

MMM 1.1278679617009995
AOS 1.1265954331040997
ABT 1.126702552561579
ABBV 1.1280244098784207
ACN 1.125948188208405
ATVI 1.1268646646355902
AYI 1.1265426281475344
ADBE 1.1259989779792061
AAP 1.1266267333566373
AMD 1.1262194458198616
AES 1.1265343229340175
AMG 1.1266108951233051
AFL 1.1265431504062406
A 1.1266510816075086
APD 1.1263231556311408
AKAM 1.1264682264676207
ALK 1.1265982998263655
ALB 1.1266500279586433
ARE 1.126461498213534
ALXN 1.1264028407699838
ALGN 1.1262722073586962
ALLE 1.1265201093491988
AGN 1.1268520748203739
ADS 1.126630037255528
LNT 1.1265324679437174
ALL 1.1264930827242186
GOOGL 1.1281165237196873
GOOG 1.1279685499587213
MO 1.1269755436462936
AMZN 1.1242509373108336
AEE 1.1265189598267014
AAL 1.1267195387410172
AEP 1.1264258360222332
AXP 1.126111918963048
AIG 1.126240853178467
AMT 1.1260417570925143
AWK 1.1264403177755322
AMP 1.1263910025272683
ABC 1.1265740057172264
AME 1.1264710992652816
AMGN 1.1279714051899012
APH 1.126565557619047
APC 1.1260238038969532
ADI 1.126

In [93]:
#sp500.sp500_df.sort_values(['Sector','Market Cap'])

sp500.sp500_df.sort_values(['Market Cap'],ascending=False).groupby('Sector').head(5).sort_values(['Sector','Market Cap']).groupby('Sector').sum()['Market Cap']

Sector
Consumer Discretionary        1.385648e+12
Consumer Staples              1.015849e+12
Energy                        7.683033e+11
Financials                    1.443666e+12
Health Care                   1.114031e+12
Industrials                   6.963972e+11
Information Technology        3.485269e+12
Materials                     3.432457e+11
Real Estate                   2.171291e+11
Telecommunication Services    4.530427e+11
Utilities                     2.475150e+11
Name: Market Cap, dtype: float64

In [83]:
sp500.sp500_df.groupby('Sector').sum()['Market Cap']

Sector
Consumer Discretionary        3.213563e+12
Consumer Staples              2.087076e+12
Energy                        1.357314e+12
Financials                    3.442649e+12
Health Care                   3.244359e+12
Industrials                   2.411541e+12
Information Technology        6.727122e+12
Materials                     6.923003e+11
Real Estate                   6.253157e+11
Telecommunication Services    4.530427e+11
Utilities                     6.116326e+11
Name: Market Cap, dtype: float64

In [74]:
for w in sorted(output, key=output.get, reverse=True):
    print( w, round(output[w],5), sp500.sp500_df[sp500.sp500_df['Symbol']==w]['Market Cap'].tolist()[0]/1e9)

INTC 1.12846 211.536
BRK.B 1.12825 261.401203633
PFE 1.12819 208.505541949
DWDP 1.12813 165.203312427
GOOGL 1.12812 733.823966137
ABBV 1.12802 181.386347059
AMGN 1.12797 128.13334
GOOG 1.12797 728.53555814
WFC 1.12796 281.463620775
UNH 1.12792 218.834811333
MMM 1.12787 138.721055226
BIIB 1.12767 69.157726427
WBA 1.12763 70.862541911
KHC 1.12759 89.618309338
CVS 1.12748 75.323141722
BMY 1.12742 102.50650196
XOM 1.12734 326.14866
CI 1.12725 47.68091048
KO 1.12719 189.855335601
LLY 1.12716 84.475986228
JNJ 1.12716 353.062464971
VZ 1.12713 208.092277044
COP 1.12712 65.48246241
OXY 1.12712 53.467692395
REGN 1.12709 35.950369241
MRK 1.12708 152.24153034
CAT 1.12707 91.822049046
UPS 1.12707 96.436356833
AVGO 1.12705 92.791974933
MDT 1.12699 110.1070623
MO 1.12698 126.985101434
LOW 1.12697 82.909678852
DE 1.12697 52.186628646
GILD 1.12696 108.106822109
BK 1.12694 56.083904906
HUM 1.12694 36.973617235
EOG 1.12693 61.164030149
MPC 1.12693 31.63374
CTL 1.12689 18.237196861
CVX 1.12689 218.9788201

In [51]:
from random import sample
sample(sp500.sp500_df['Symbol'].tolist(),1)

['WMT']

In [ ]:
1.1265529408262145
ALK

In [34]:
counter=0
for i in z.stock_weights:
    if z.stock_weights[i]!=0:
        counter+=1
    else:
        print(i)
print(counter)

AET
ANDV
BF.B
CA
CBG
CSRA
DPS
EVHC
ESRX
FITB
GGP
LUK
MON
NFX
PX
PCLN
COL
SCG
SNI
TWX
FOXA
FOX
HCN
WYN
XL
480


In [ ]:
performance_results=[]
for i in range (100):
    performance_results.append(random_perf(50))

In [ ]:
import statistics
print(max(performance_results))
print(min(performance_results))

print(statistics.mean(performance_results))
print(statistics.stdev(performance_results))

In [ ]:
10841.229755636177
7526.546772302236
9072.111642584901
621.0216097915476

10951.0677977124
7476.093964348738
9091.096682826419
663.6401236122267

In [ ]:
def avg_list(inputs):
    for i in inputs:
        total+=i
    return total/len(inputs)

In [21]:
import api_key

In [23]:
spy=Stock('SPY','E4XLANTEHDNE0HT9')

In [26]:
spy_shares=1/float(json.loads(spy.api_output.content)['Time Series (Daily)']['2019-01-02']['5. adjusted close'])
spy_shares*float(json.loads(spy.api_output.content)['Time Series (Daily)']['2019-05-28']['5. adjusted close'])

1.124702464710277

In [ ]:
pd.DataFrame(json.loads(sp500.sp500_prices['DIS'].api_output.content)['Time Series (Daily)']).transpose()

In [ ]:
port=Portfolio(sample_stocks['Symbol'].tolist())

In [ ]:
print(port.Stocks[5].symbol,port.Stocks[5].api_output.content)

In [ ]:
port.total_portfolio_df()

In [ ]:
stock1=Stock('DIS',api_key)
stock2=Stock('T',api_key)
combined=stock1.get_stock_df().join(stock2.get_stock_df(),lsuffix=' '+stock1.symbol,rsuffix=' '+stock2.symbol)

In [ ]:
stock1.get_stock_df()+stock2.get_stock_df()

In [ ]:
#combined

In [ ]:
combined[['5. adjusted close '+stock1.symbol,'5. adjusted close '+stock2.symbol]].astype(float).corr()

In [ ]:


stock_list=sp500_df.sample(5)['Symbol'].tolist()
index=Portfolio(stock_list)
index.Stocks[0].get_stock_df()

In [ ]:
z=Index()

In [ ]:
z.

In [ ]:
z=Stock('DIS',api_key)
z.get_stock_df()